<a href="https://colab.research.google.com/github/Vladimir8243/ipyparallel/blob/master/Copy_of_Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: August 28th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [5]:
!nvidia-smi

Sat Sep  7 19:41:18 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [6]:
import requests
full_text = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt').text
text = full_text.split('THE SONNETS')[1]


print(text[:100])



by William Shakespeare



                     1
  From fairest creatures we desire increase,
  Th


In [7]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 290Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 109Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 595Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 231Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 375Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 170Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 168Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [8]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "shakespeare.txt"
#file_name = text
#text_file = open(file_name, 'w+')
#text_file.write(text)
#text_file.close()




If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [10]:
file_name[:10]

'shakespear'

In [0]:
!ls

In [0]:
gpt2.copy_file_from_gdrive(file_name)

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

# New Section

In [13]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.25s/it]


dataset has 1847425 tokens
Training...
[10 | 28.44] loss=3.04 avg=3.04
[20 | 50.78] loss=2.99 avg=3.02
[30 | 73.91] loss=2.80 avg=2.95
[40 | 98.21] loss=2.89 avg=2.93
[50 | 122.73] loss=3.32 avg=3.01
[60 | 146.48] loss=2.57 avg=2.93
[70 | 170.37] loss=2.95 avg=2.94
[80 | 194.67] loss=2.68 avg=2.90
[90 | 218.98] loss=2.79 avg=2.89
[100 | 243.22] loss=2.73 avg=2.87
[110 | 267.39] loss=2.73 avg=2.86
[120 | 291.47] loss=2.77 avg=2.85
[130 | 315.61] loss=2.68 avg=2.84
[140 | 339.81] loss=2.66 avg=2.82
[150 | 364.08] loss=2.66 avg=2.81
[160 | 388.37] loss=2.97 avg=2.82
[170 | 412.62] loss=3.03 avg=2.84
[180 | 436.80] loss=2.94 avg=2.84
[190 | 461.00] loss=2.32 avg=2.81
[200 | 485.25] loss=2.40 avg=2.79
======== SAMPLE 1 ========
 thee of him
     How you, if he be such, have no more in him.
     You are a kind of fool, sir, which is a kind of mirth and a kind of
     foolishness; and so, unless you be such, you are fool enough of 'em.
     There's that which I should not say, which I cannot 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [6]:
gpt2.generate(sess, run_name='run1')

                                         Exit ARVIRAGUS
    You are nothing like her that thought on her.
    But what else?
    You were a better creature if you had
    Had rather have been a man than a woman.
    You were more like a child than a man,
    Than had been your father: 'Tis strange, I know
    That women are children of men.
    Your mother, you, and you are children of women.
    You were married, were you not?  
    Why, how else?
  QUEEN KATHARINE. You were not as old as I,
    But as old as your father was,
    And as old as he was in his age.
    But what else?
  ARVIRAGUS. You were young and young in your age.
  QUEEN KATHARINE. And so old as your father was,
    That being no man, being a woman,
    Whose age was not yet in your age,
    Were not the age of your father.
  QUEEN KATHARINE. But I say all age is so.
    Therefore you were not as old as me as yours;
    And so, being no man, being a woman,
    Being old, were you no woman.
    I am not a woman, nor a

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [7]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD. I'll give thee three times much
    To be gracious, and give thee one great thanks;
    For thou hast done me right.
    The Prince, and thy brother, are come
    And have made me a gracious Prince;
    But, since thou hast done me wrong,
    I'll give thee away; for I will not
    Give away the Prince to th' undeserving
    Of my love, that art so much in pity.
    Now, good Prince, will I be King again?
    Give me thy hand; for, if I live,
    The Duke, living, shall be my eldest son.
    Thus I go, and do my best to win him  
    As I do to win my brother.
    Speak, good Prince; speak no more of me.
    Let me not bid thee thank me for it;
    Tell me, Prince, if thou wilt be King again,
    What thou wilt to be King of England;
 
LORD. Pray you, my lord, what's this?
    No more than a tabor or a pile of hay
    Will make me beggar.
  SECOND LORD. You may say that, if he were a man,
    He would fetch you.
  FIRST LORD. A kind of a man,
    A kind of gentleman,
    A kind o

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M "large" model.

In [2]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 465Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 108Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 739Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:11, 259Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 420Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 201Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 186Mit/s]                                                       


In [3]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/774M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


In [4]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The secret of life is not to make a big deal of it. It's not to try to impress the world. It's to just be. And it's to just be. It's to be.

When you think about it, it's very simple. Life is about being. And that's how we are all living it. We are all living it in the moment.

So when I look at this, and I see this picture of my child, and I think about how amazing and beautiful
The secret of life is in the structure of the cells, and the complexity of the way they function is the result of their DNA and RNA.

We live in an environment that is very complex, and the complexity of life is based on a very simple system: DNA and RNA.

The human genome is the most complex thing in the universe.

The human genome is the most complex thing in the universe.

The human genome is the most complex thing in the universe.

The human genome
The secret of life is in the presence of a large enough number of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.